In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from imblearn.combine import SMOTEENN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import roc_auc_score, recall_score, precision_score, f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from xgboost import XGBClassifier

### Read data file 


In [2]:
def read_data(file):
    """Returns a np array from file. Each row is padded with -999.0 in order
    to have the same length as the row with the maximun number of NMR"""

    X = []
    y = []
    for line in file:
        spectra = [float(nmr) for nmr in line.split(", ")[:-1]]
        X.append(np.array(spectra))
        y.append(np.array(int(line.split(", ")[-1].strip())))

    X = np.array(X, dtype=object)
    y = np.array(y)

    return (X, y)

In [3]:
file = open("../data/alkaloids.csv")

In [4]:
X, y = read_data(file)

In [5]:
X

array([[11.2, 17.0, 25.0, ..., -999.0, -999.0, -999.0],
       [17.3, 30.9, 48.5, ..., -999.0, -999.0, -999.0],
       [14.1, 14.8, 16.0, ..., -999.0, -999.0, -999.0],
       ...,
       [12.9, 17.9, 18.3, ..., -999.0, -999.0, -999.0],
       [7.0, 15.5, 18.9, ..., -999.0, -999.0, -999.0],
       [15.4, 17.1, 17.3, ..., -999.0, -999.0, -999.0]], dtype=object)

In [6]:
X.shape

(17376, 37)

Thus, the max number of NMR spectra = 37

#### Now, we get the alkaloids spectra with 37 spectra:

In [7]:
# Get positive indexes
pos_idcs = y == 1

In [8]:
X_pos = X[pos_idcs]  # Alkaloids NMR

In [9]:
len(X_pos)

137

In [10]:
[x for x in X_pos if not -999.0 in x]

[array([15.6, 18.6, 19.5, 20.5, 25.8, 26.5, 27.3, 27.4, 27.7, 29.4, 30.2,
        36.5, 37.0, 37.6, 42.4, 49.9, 52.3, 52.6, 61.8, 64.7, 65.8, 71.4,
        71.5, 73.6, 78.0, 108.3, 109.3, 112.3, 116.3, 121.1, 123.4, 129.7,
        131.3, 138.9, 141.1, 149.2, 152.0], dtype=object),
 array([18.6, 19.0, 19.7, 20.3, 21.4, 24.7, 26.9, 28.9, 30.6, 31.1, 35.1,
        43.6, 47.0, 50.1, 52.7, 58.8, 61.9, 66.1, 66.3, 72.0, 72.4, 74.7,
        76.1, 78.2, 81.0, 107.1, 111.6, 111.9, 120.6, 122.0, 124.6, 125.8,
        133.3, 139.7, 143.3, 149.5, 154.4], dtype=object)]

In the inference time, if the input has more than 37 spectra it will be predicted as non-alkaloid. 

### Model evaluation strategy:
* Validation set = 30%
* 70% for resampling and hyperparameter search (CV 5 fold)

In [11]:
kf = 5

In [12]:
test_size = 0.3

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, random_state=0, stratify=y
)

### Resampling strategy:
 1.- Random under sampling of the majority class
 
 2.- Combination of over and under sampling using SMOTEENN

We choose 10% (sampling strategy = 0.1) of positive class as the final imbalance as we do not want to affect too much the original distribution of the classes.

In [14]:
under_sampler = RandomUnderSampler(sampling_strategy=0.1, random_state=0)

In [15]:
X_under, y_under = under_sampler.fit_resample(X_train, y_train)

In [16]:
X_under.shape

(1056, 37)

In [17]:
resampler = SMOTEENN(sampling_strategy=0.1, random_state=0)

In [18]:
X_res, y_res = resampler.fit_resample(X_train, y_train)

In [19]:
# new number of positive samples
len(y_res[y_res == 1])

1109

In [20]:
X_res.shape

(12936, 37)

In [21]:
space = {
    "max_depth": hp.quniform("max_depth", 3, 18, 1),
    "reg_alpha": hp.uniform("reg_alpha", 0, 10),
    "reg_lambda": hp.uniform("reg_lambda", 0, 10),
    "min_child_weight": hp.quniform("min_child_weight", 0, 5, 1),
    "n_estimators": hp.quniform("n_estimators", 80, 320, 20),
    "seed": hp.choice("seed", [22, 44, 66]),
    "eta": hp.uniform("eta", 0.1, 0.9),
}

In [22]:
def objective(space, kf=kf, X=X_under, y=y_under):

    clf = XGBClassifier(
        max_depth=int(space["max_depth"]),
        reg_alpha=space["reg_alpha"],
        reg_lambda=space["reg_lambda"],
        min_child_weight=int(space["min_child_weight"]),
        n_estimators=int(space["n_estimators"]),
        seed=space["seed"],
        eta=space["eta"],
    )

    cv = StratifiedKFold(random_state=22, n_splits=kf, shuffle=True)
    score = cross_val_score(clf, X, y, cv=cv, scoring="recall", n_jobs=-1).mean()
    print(f"SCORE: {score}")

    return {"loss": -score, "status": STATUS_OK}

In [23]:
trials_under = Trials()

best_hyperparams_under = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials_under,
    rstate=np.random.default_rng(789)
)

SCORE: 0.4068421052631579                             
SCORE: 0.2831578947368421                                                        
SCORE: 0.2094736842105263                                                        
SCORE: 0.29315789473684206                                                       
SCORE: 0.3857894736842105                                                        
SCORE: 0.3757894736842105                                                        
SCORE: 0.16736842105263156                                                       
SCORE: 0.3031578947368421                                                        
SCORE: 0.36473684210526314                                                       
SCORE: 0.16684210526315787                                                       
SCORE: 0.18842105263157893                                                        
SCORE: 0.26105263157894737                                                        
SCORE: 0.3136842105263158                

In [24]:
trials_under.best_trial

{'state': 2,
 'tid': 0,
 'spec': None,
 'result': {'loss': -0.4068421052631579, 'status': 'ok'},
 'misc': {'tid': 0,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'eta': [0],
   'max_depth': [0],
   'min_child_weight': [0],
   'n_estimators': [0],
   'reg_alpha': [0],
   'reg_lambda': [0],
   'seed': [0]},
  'vals': {'eta': [0.43911010940117146],
   'max_depth': [15.0],
   'min_child_weight': [2.0],
   'n_estimators': [120.0],
   'reg_alpha': [0.0747528345435633],
   'reg_lambda': [5.147611893010373],
   'seed': [0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2022, 9, 29, 4, 20, 40, 55000),
 'refresh_time': datetime.datetime(2022, 9, 29, 4, 20, 44, 689000)}

#### Now, using the resampled set with over and under sampling

In [25]:
def objective(space, kf=kf, X=X_res, y=y_res):

    clf = XGBClassifier(
        max_depth=int(space["max_depth"]),
        reg_alpha=space["reg_alpha"],
        reg_lambda=space["reg_lambda"],
        min_child_weight=int(space["min_child_weight"]),
        n_estimators=int(space["n_estimators"]),
        seed=space["seed"],
        eta=space["eta"],
    )

    cv = StratifiedKFold(random_state=22, n_splits=kf, shuffle=True)
    score = cross_val_score(clf, X, y, cv=cv, scoring="recall", n_jobs=-1).mean()
    print(f"SCORE: {score}")

    return {"loss": -score, "status": STATUS_OK}

In [26]:
trials_res = Trials()

best_hyperparams_res = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=20,
    trials=trials_res,
    rstate=np.random.default_rng(789)
)

SCORE: 0.9612123435652847                             
SCORE: 0.9098406098406098                                                        
SCORE: 0.9025967143614203                                                        
SCORE: 0.938665362194774                                                         
SCORE: 0.954001059883413                                                         
SCORE: 0.9314540785129021                                                        
SCORE: 0.8917614447026212                                                        
SCORE: 0.9269414210590681                                                        
SCORE: 0.9513105866047044                                                        
SCORE: 0.8908605438017203                                                        
SCORE: 0.893591781827076                                                          
SCORE: 0.907113448289919                                                          
SCORE: 0.9341649341649342                

In [27]:
trials_res.best_trial

{'state': 2,
 'tid': 0,
 'spec': None,
 'result': {'loss': -0.9612123435652847, 'status': 'ok'},
 'misc': {'tid': 0,
  'cmd': ('domain_attachment', 'FMinIter_Domain'),
  'workdir': None,
  'idxs': {'eta': [0],
   'max_depth': [0],
   'min_child_weight': [0],
   'n_estimators': [0],
   'reg_alpha': [0],
   'reg_lambda': [0],
   'seed': [0]},
  'vals': {'eta': [0.43911010940117146],
   'max_depth': [15.0],
   'min_child_weight': [2.0],
   'n_estimators': [120.0],
   'reg_alpha': [0.0747528345435633],
   'reg_lambda': [5.147611893010373],
   'seed': [0]}},
 'exp_key': None,
 'owner': None,
 'version': 0,
 'book_time': datetime.datetime(2022, 9, 29, 4, 21, 34, 941000),
 'refresh_time': datetime.datetime(2022, 9, 29, 4, 21, 48, 611000)}

#### The 2 datasets and the 2 hyperparameters search results will be employed for training 2 new XGBoost models (`XGB_under` and `XGB_res`). Then, these models will be tested on the hold-out test set.

`XGB_under`: model trained with the undersampled dataset `X_under`

`XGB_res`: model trained with the resampled dataset `X_res`


In [28]:
#hyperparams undersampled dataset
params_under = trials_under.best_trial['misc']['vals']

In [29]:
params_under

{'eta': [0.43911010940117146],
 'max_depth': [15.0],
 'min_child_weight': [2.0],
 'n_estimators': [120.0],
 'reg_alpha': [0.0747528345435633],
 'reg_lambda': [5.147611893010373],
 'seed': [0]}

In [30]:
#Get the int and floats from the lists
for key, val in params_under.items():
    params_under[key] = val[0]

In [31]:
params_under

{'eta': 0.43911010940117146,
 'max_depth': 15.0,
 'min_child_weight': 2.0,
 'n_estimators': 120.0,
 'reg_alpha': 0.0747528345435633,
 'reg_lambda': 5.147611893010373,
 'seed': 0}

In [32]:
# Change floats to int
params_under['max_depth'] = int(params_under['max_depth'])
params_under['min_child_weight'] = int(params_under['min_child_weight'])
params_under['n_estimators'] = int(params_under['n_estimators'])

In [33]:
params_under

{'eta': 0.43911010940117146,
 'max_depth': 15,
 'min_child_weight': 2,
 'n_estimators': 120,
 'reg_alpha': 0.0747528345435633,
 'reg_lambda': 5.147611893010373,
 'seed': 0}

In [34]:
XGB_under = XGBClassifier(**params_under)

In [35]:
XGB_under.fit(X_under, y_under)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eta=0.43911010940117146, eval_metric=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.4391101, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=15, max_leaves=0,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=120, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0.0747528345435633, ...)

In [36]:
y_under_pred = XGB_under.predict(X_test)

In [37]:
recall_score(y_under_pred, y_test)

0.12359550561797752

In [38]:
precision_score(y_under_pred, y_test)

0.2682926829268293

In [39]:
f1_score(y_under_pred, y_test)

0.16923076923076924

In [40]:
roc_auc_score(y_under_pred, y_test)

0.5588703523406048

In [41]:
#hyperparams resampled dataset
params_res = trials_res.best_trial['misc']['vals']

In [42]:
params_res

{'eta': [0.43911010940117146],
 'max_depth': [15.0],
 'min_child_weight': [2.0],
 'n_estimators': [120.0],
 'reg_alpha': [0.0747528345435633],
 'reg_lambda': [5.147611893010373],
 'seed': [0]}

In [43]:
#Get the int and floats from the lists
for key, val in params_res.items():
    params_res[key] = val[0]

In [44]:
params_res

{'eta': 0.43911010940117146,
 'max_depth': 15.0,
 'min_child_weight': 2.0,
 'n_estimators': 120.0,
 'reg_alpha': 0.0747528345435633,
 'reg_lambda': 5.147611893010373,
 'seed': 0}

In [45]:
# Change floats to int
params_res['max_depth'] = int(params_res['max_depth'])
params_res['min_child_weight'] = int(params_res['min_child_weight'])
params_res['n_estimators'] = int(params_res['n_estimators'])

In [46]:
params_res

{'eta': 0.43911010940117146,
 'max_depth': 15,
 'min_child_weight': 2,
 'n_estimators': 120,
 'reg_alpha': 0.0747528345435633,
 'reg_lambda': 5.147611893010373,
 'seed': 0}

In [47]:
XGB_res = XGBClassifier(**params_res)

In [48]:
XGB_res.fit(X_res, y_res)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eta=0.43911010940117146, eval_metric=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.4391101, max_bin=256,
              max_cat_to_onehot=4, max_delta_step=0, max_depth=15, max_leaves=0,
              min_child_weight=2, missing=nan, monotone_constraints='()',
              n_estimators=120, n_jobs=0, num_parallel_tree=1, predictor='auto',
              random_state=0, reg_alpha=0.0747528345435633, ...)

In [49]:
y_res_pred = XGB_res.predict(X_test)

In [50]:
recall_score(y_res_pred, y_test)

0.3103448275862069

In [51]:
precision_score(y_res_pred, y_test)

0.21951219512195122

In [52]:
f1_score(y_res_pred, y_test)

0.2571428571428571

In [53]:
roc_auc_score(y_res_pred, y_test)

0.652085994040017